In [4]:
#IMPORTING LIBRARIES
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import matplotlib.ticker as ticker
import pickle

import os

In [5]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
scratchDirectory='/home/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Figures/Campaign_Data'

In [6]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data,parcel] = GetData(dataDirectory, parcelDirectory)

In [ ]:
dir = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

In [ ]:
n_bins=10
hours_per_timestep=60/5
previous_hours=11
previous_times=-hours_per_timestep*previous_hours

In [ ]:
# Define land and ocean slices
fraction=2/8
land = slice(int(len(data['xh'])*fraction), int(len(data['xh'])))
ocean = slice(0, int(len(data['xh'])*fraction))

In [ ]:
def GetData1(var_name):
    land_data = data[var_name].isel(xh=land).data#.mean(dim=['xh', 'yh'])
    ocean_data = data[var_name].isel(xh=ocean).data#.mean(dim=['xh', 'yh'])
    
    land_data_mean = np.mean(land_data,axis=(1,2))
    ocean_data_mean = np.mean(ocean_data,axis=(1,2))
    
    return land_data_mean,ocean_data_mean

def GetData2(var_name):
    land_data = data[var_name].isel(xh=land,zh=0).data#.mean(dim=['xh', 'yh'])
    ocean_data = data[var_name].isel(xh=ocean,zh=0).data#.mean(dim=['xh', 'yh'])
    
    land_data_mean = np.mean(land_data,axis=(1,2))
    ocean_data_mean = np.mean(ocean_data,axis=(1,2))
    
    return land_data_mean,ocean_data_mean

def SaveLandOceanData(data_list, filename=dir+'Figures/campaign_data/'+f'land_ocean_data_{res}_{t_res}.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump(data_list, f)
    print(f"Saved land and ocean data to '{filename}'")

def LoadLandOceanData(filename=dir+'Figures/campaign_data/'+f'land_ocean_data_{res}_{t_res}.pkl'):
    with open(filename, 'rb') as f:
        data_list = pickle.load(f)
    return data_list[0], data_list[1]

In [ ]:
# Create figure and axes
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
fig.suptitle('Surface Area Means')

# Surface Temperature Plot
ax = axes[0, 0]
var_name='tsk'
[land_data_mean,ocean_data_mean] = GetData1(var_name)
SaveLandOceanData([land_data_mean,ocean_data_mean], filename=dir+'Figures/campaign_data/'+f'land_ocean_data_{res}_{t_res}.pkl')
ax.plot(land_data_mean, color='brown', label='land')
ax.plot(ocean_data_mean, color='blue', label='ocean')
ax.set_title('Surface Temperature (K)')
ax.set_xlabel('Timestep')
ax.set_ylabel('Temp (K)')
ax.legend(loc='lower left')
ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=n_bins))  # Set Y-Ticks
ax.set_xlim(left=previous_times) #matching data diurnal cycle

# # QV Flux Plot
# ax = axes[0, 1]
# var_name='qvflux'
# [land_data_mean,ocean_data_mean] = GetData1(var_name)
# ax.plot(land_data_mean, color='brown', label='land')
# ax.plot(ocean_data_mean, color='blue', label='ocean')
# ax.set_title('QV Flux (K)')
# ax.set_xlabel('Timestep')
# ax.set_ylabel(r'qvflux ($g\ g^{-1} m\ s^{-1}$)')
# ax.legend(loc='lower left')
# ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=n_bins))  # Set Y-Ticks
# ax.set_xlim(left=previous_times) #matching data diurnal cycle

# # Theta Flux Plot
# ax = axes[1, 0]
# var_name='thflux'
# [land_data_mean,ocean_data_mean] = GetData1(var_name)
# ax.plot(land_data_mean, color='brown', label='land')
# ax.plot(ocean_data_mean, color='blue', label='ocean')
# ax.set_title('Theta Flux (K)')
# ax.set_xlabel('Timestep')
# ax.set_ylabel(r'thflux ($K\ m\ s^{-1}$)')
# ax.legend(loc='lower left')
# ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=n_bins))  # Set Y-Ticks
# ax.set_xlim(left=previous_times) #matching data diurnal cycle

# # U Wind Component Plot
# ax = axes[1, 1]
# var_name='uinterp'
# [land_data_mean,ocean_data_mean] = GetData2(var_name)
# ax.plot(land_data_mean, color='brown', label='land')
# ax.plot(ocean_data_mean, color='blue', label='ocean')
# ax.set_title(r'U ($m\ s^{-1}$)')
# ax.set_xlabel('Timestep')
# ax.set_ylabel(r'u ($m\ s^{-1}$)')
# ax.legend(loc='lower left')
# ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=n_bins))  # Set Y-Ticks
# ax.set_xlim(left=previous_times) #matching data diurnal cycle